In [1]:
!pip install groq python-dotenv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.6 MB/s eta 0:00:00


In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [3]:
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

MODEL = "mixtral-8x7b-32768"

# ── Expert configurations ──────────────────────────────────────────────────────
MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a senior software engineer and technical support specialist. "
            "You are rigorous, precise, and code-focused. When answering questions: "
            "1) Diagnose the root cause clearly. "
            "2) Provide corrected code snippets with inline comments. "
            "3) Explain WHY the bug occurred. "
            "4) Suggest best practices to prevent recurrence. "
            "Use markdown code blocks for all code."
        ),
        "temperature": 0.7,
    },
    "billing": {
        "system_prompt": (
            "You are an empathetic and professional billing support specialist. "
            "You are policy-driven, calm, and solution-oriented. When answering: "
            "1) Acknowledge the customer's frustration first. "
            "2) Explain the relevant billing policy clearly. "
            "3) Outline the exact steps to resolve their issue (refund, dispute, etc.). "
            "4) Reassure them with a clear timeline. "
            "Never make promises outside standard policy."
        ),
        "temperature": 0.7,
    },
    "general": {
        "system_prompt": (
            "You are a friendly and helpful general customer support assistant. "
            "Be conversational, warm, and concise. Handle casual questions, "
            "onboarding queries, and anything that doesn't fit technical or billing topics. "
            "If the question is outside your knowledge, say so honestly and offer to escalate."
        ),
        "temperature": 0.7,
    },
}

# ── Router ─────────────────────────────────────────────────────────────────────
def route_prompt(user_input: str) -> str:
    """Classify user intent and return the category name only."""
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0,  # Deterministic classification
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an intent classifier for a customer support system. "
                    "Classify the user's message into exactly ONE of these categories:\n"
                    "  - technical  (bugs, errors, code issues, API problems, crashes)\n"
                    "  - billing    (charges, refunds, subscriptions, invoices, payments)\n"
                    "  - general    (everything else)\n\n"
                    "Rules:\n"
                    "1. Return ONLY the single lowercase category word.\n"
                    "2. No punctuation, no explanation, no extra words.\n"
                    "3. If unsure, return: general"
                ),
            },
            {"role": "user", "content": user_input},
        ],
    )
    category = response.choices[0].message.content.strip().lower()
    # Safety fallback in case the model adds extra text
    for valid in MODEL_CONFIG:
        if valid in category:
            return valid
    return "general"

# ── Orchestrator ───────────────────────────────────────────────────────────────
def process_request(user_input: str) -> dict:
    """Full MoE pipeline: route → select expert → generate response."""
    print(f"\n📨 User: {user_input}")

    # Step 1: Route
    category = route_prompt(user_input)
    print(f"🔀 Router decision: [{category.upper()} EXPERT]")

    # Step 2: Select expert config
    config = MODEL_CONFIG[category]

    # Step 3: Generate expert response
    response = client.chat.completions.create(
        model=MODEL,
        temperature=config["temperature"],
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input},
        ],
    )
    answer = response.choices[0].message.content.strip()

    print(f"🤖 Expert Response:\n{answer}\n")
    print("─" * 60)

    return {"category": category, "response": answer}

In [8]:
# Fix: update to current supported model
MODEL = "llama-3.3-70b-versatile"

In [9]:
test_queries = [
    "My Python script is throwing an IndexError on line 5.",
    "I was charged twice for my subscription this month.",
    "What features does your Pro plan include?",
]

for query in test_queries:
    process_request(query)


📨 User: My Python script is throwing an IndexError on line 5.
🔀 Router decision: [TECHNICAL EXPERT]
🤖 Expert Response:
### Diagnosing the Issue
To accurately diagnose the issue, I would need to see the code. However, I can provide a general example of how an IndexError might occur and how to fix it.

### Example Code with Error
```python
def example_function():
    my_list = [1, 2, 3]
    # Attempting to access an index that does not exist
    print(my_list[3])  # This will throw an IndexError

example_function()
```

### Corrected Code
```python
def example_function():
    my_list = [1, 2, 3]
    # Check if the index exists before attempting to access it
    index = 3
    if index < len(my_list):
        print(my_list[index])  # This will print the value at the index if it exists
    else:
        print(f"Index {index} does not exist in the list.")

example_function()
```

### Explanation of the Bug
The bug occurred because the code attempted to access an index in a list that does no

In [10]:
import random

# ── Mock tool ──────────────────────────────────────────────────────────────────
def mock_fetch_bitcoin_price() -> dict:
    """Simulates an API call to a crypto price feed."""
    price = random.uniform(60000, 75000)
    return {
        "symbol": "BTC/USD",
        "price": round(price, 2),
        "change_24h": round(random.uniform(-5, 5), 2),
        "source": "MockCryptoAPI v1.0",
    }

# ── Extended router with 'crypto' category ────────────────────────────────────
MODEL_CONFIG["crypto"] = {
    "system_prompt": "You are a crypto data assistant. Report prices clearly and add a brief market note.",
    "temperature": 0.5,
}

def route_prompt_extended(user_input: str) -> str:
    """Router extended with crypto detection."""
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": (
                    "Classify the user's message into ONE of:\n"
                    "  - technical\n  - billing\n  - crypto  (price, value of Bitcoin/ETH/crypto)\n  - general\n\n"
                    "Return ONLY the single lowercase word. No punctuation."
                ),
            },
            {"role": "user", "content": user_input},
        ],
    )
    category = response.choices[0].message.content.strip().lower()
    for valid in MODEL_CONFIG:
        if valid in category:
            return valid
    return "general"

def process_request_extended(user_input: str) -> dict:
    """MoE pipeline with tool-use branch for crypto queries."""
    print(f"\n📨 User: {user_input}")
    category = route_prompt_extended(user_input)
    print(f"🔀 Router decision: [{category.upper()} EXPERT]")

    # Tool-use branch
    if category == "crypto":
        data = mock_fetch_bitcoin_price()
        tool_context = (
            f"Live market data fetched:\n"
            f"  Symbol: {data['symbol']}\n"
            f"  Price:  ${data['price']:,.2f}\n"
            f"  24h Change: {data['change_24h']}%\n"
            f"  Source: {data['source']}"
        )
        config = MODEL_CONFIG["crypto"]
        response = client.chat.completions.create(
            model=MODEL,
            temperature=config["temperature"],
            messages=[
                {"role": "system", "content": config["system_prompt"]},
                {"role": "user", "content": f"{user_input}\n\n[TOOL RESULT]\n{tool_context}"},
            ],
        )
        answer = response.choices[0].message.content.strip()
    else:
        config = MODEL_CONFIG[category]
        response = client.chat.completions.create(
            model=MODEL,
            temperature=config["temperature"],
            messages=[
                {"role": "system", "content": config["system_prompt"]},
                {"role": "user", "content": user_input},
            ],
        )
        answer = response.choices[0].message.content.strip()

    print(f"🤖 Expert Response:\n{answer}\n")
    print("─" * 60)
    return {"category": category, "response": answer}

# Test the bonus
process_request_extended("What is the current price of Bitcoin?")
process_request_extended("I have a KeyError in my dictionary lookup.")


📨 User: What is the current price of Bitcoin?
🔀 Router decision: [CRYPTO EXPERT]
🤖 Expert Response:
The current price of Bitcoin is $70,053.76. The market seems relatively stable, with a minor 24-hour decline of 0.19%, indicating a slight bearish sentiment but overall calm trading activity.

────────────────────────────────────────────────────────────

📨 User: I have a KeyError in my dictionary lookup.
🔀 Router decision: [TECHNICAL EXPERT]
🤖 Expert Response:
### Diagnosing the Root Cause
The `KeyError` exception in Python is typically raised when a key is not found in a dictionary. To diagnose this issue, we need to examine the code snippet that's causing the error.

### Example Code with Error
```python
# Example dictionary
data = {
    'name': 'John',
    'age': 30
}

# Attempting to access a non-existent key
print(data['city'])  # This line will raise a KeyError
```

### Corrected Code Snippet
To avoid the `KeyError`, you can use the `get()` method, which returns `None` by default 

{'category': 'technical',
 'response': "### Diagnosing the Root Cause\nThe `KeyError` exception in Python is typically raised when a key is not found in a dictionary. To diagnose this issue, we need to examine the code snippet that's causing the error.\n\n### Example Code with Error\n```python\n# Example dictionary\ndata = {\n    'name': 'John',\n    'age': 30\n}\n\n# Attempting to access a non-existent key\nprint(data['city'])  # This line will raise a KeyError\n```\n\n### Corrected Code Snippet\nTo avoid the `KeyError`, you can use the `get()` method, which returns `None` by default if the key is not found:\n```python\n# Example dictionary\ndata = {\n    'name': 'John',\n    'age': 30\n}\n\n# Using get() to avoid KeyError\ncity = data.get('city')  # Returns None if 'city' is not found\nprint(city)  # Output: None\n\n# Alternatively, you can provide a default value\ncity = data.get('city', 'Not specified')  # Returns 'Not specified' if 'city' is not found\nprint(city)  # Output: Not s